In [1]:
import duckdb
import os
from dotenv import load_dotenv

load_dotenv()
con = duckdb.connect()

try:
    con.sql(f"""
        INSTALL postgres;
        LOAD postgres;
        ATTACH 'dbname={os.getenv("PGDATABASE")} user={os.getenv("PGUSER")} password={os.getenv("PGPASSWORD")} \
            host={os.getenv("PGHOST")} port={os.getenv("PGPORT")}' AS remote_mimic (TYPE POSTGRES);
    """)
    print("DuckDB attached to remote PostgreSQL successfully.")

except Exception as e:
    print(f"Error attaching PostgreSQL: {e}")

DuckDB attached to remote PostgreSQL successfully.


In [2]:
import pandas as pd

cohort_query = """
DROP TABLE IF EXISTS cohort_tab;
CREATE TABLE cohort_tab AS
WITH cohort AS (
    SELECT
        ic.subject_id,
        ic.hadm_id,
        ic.stay_id,
        ic.intime,
        ic.outtime,
        -- Calculate age precisely using anchor_year logic
        (pa.anchor_age + (date_part('year', ic.intime) - pa.anchor_year)) AS age,
        adm.hospital_expire_flag,
        adm.deathtime,
        -- Rank stays: First stay of the first admission
        ROW_NUMBER() OVER (PARTITION BY ic.subject_id ORDER BY ic.intime) AS stay_rank
    FROM
        remote_mimic.mimiciv_icu.icustays ic
    INNER JOIN 
        remote_mimic.mimiciv_hosp.patients pa ON ic.subject_id = pa.subject_id
    INNER JOIN 
        remote_mimic.mimiciv_hosp.admissions adm ON ic.hadm_id = adm.hadm_id
)
SELECT
    subject_id,
    hadm_id,
    stay_id,
    intime,
    outtime,
    age,
    hospital_expire_flag AS y_ihm
FROM
    cohort
WHERE
    stay_rank = 1               -- First ICU stay only
    AND age >= 18               -- Adults only
    -- EXCLUSION: Patient must stay at least 24h to have a full observation window
    AND date_diff('minute', intime, outtime) >= 1440 
    -- EXCLUSION: If they died, they must have died AFTER the 24h window
    -- (Prevents the model from 'seeing' the death process in the vitals)
    AND (deathtime IS NULL OR deathtime > (intime + INTERVAL '24 HOURS'))
ORDER BY
    subject_id;"""

con.sql(cohort_query)
df_cohort = con.sql("SELECT * FROM cohort_tab").df()
print(f"Cohort selection complete. Cohort size: {len(df_cohort):,} patients.")
df_cohort.head()

Cohort selection complete. Cohort size: 51,674 patients.


,subject_id,hadm_id,stay_id,intime,outtime,age,y_ihm
0,10000690,25860671,37081114,2150-11-02 19:37:00,2150-11-06 17:03:17,86,0
1,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,55,0
2,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,46,0
3,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,77,1
4,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,57,0


In [3]:
vitals_query = """
DROP TABLE IF EXISTS vitals_tab;
CREATE TABLE vitals_tab AS (
    SELECT 
        v.stay_id,
        FLOOR(date_diff('second', c.intime, v.charttime) / 3600) AS hr,
        v.heart_rate, v.sbp, v.spo2, v.temperature, v.resp_rate
    FROM cohort_tab c
    JOIN remote_mimic.mimiciv_derived.vitalsign v ON c.stay_id = v.stay_id
    WHERE v.charttime BETWEEN c.intime AND (c.intime + INTERVAL '24 HOURS')
);
"""

con.sql(vitals_query)
print("Hourly vitals extraction complete.")

labs_query = """
DROP TABLE IF EXISTS labs_tab;
CREATE TABLE labs_tab AS (
    SELECT 
        fdl.stay_id,
        fdb.lactate_max, fdl.bilirubin_total_max, fdl.creatinine_max, 
        fdl.wbc_max, fdl.glucose_max, fdl.bun_max
    FROM remote_mimic.mimiciv_derived.first_day_lab fdl
    JOIN remote_mimic.mimiciv_derived.first_day_bg_art fdb ON fdl.stay_id = fdb.stay_id
);
"""

con.sql(labs_query)
print("Lab values extraction complete.")

scores_query = """
DROP TABLE IF EXISTS scores_tab;
CREATE TABLE scores_tab AS (
    SELECT 
        s.stay_id, s.sapsii, o.oasis, so.sofa_24hours
    FROM remote_mimic.mimiciv_derived.sapsii s
    JOIN remote_mimic.mimiciv_derived.oasis o ON s.stay_id = o.stay_id
    JOIN remote_mimic.mimiciv_derived.sofa so ON s.stay_id = so.stay_id
);
"""

con.sql(scores_query)
print("Clinical scores extraction complete.")

Hourly vitals extraction complete.
Lab values extraction complete.
Clinical scores extraction complete.


In [4]:
features_query = """
DROP TABLE IF EXISTS features_tab;
CREATE TABLE features_tab AS (
    SELECT 
        v.*,
        l.lactate_max, l.creatinine_max,
        s.sapsii, s.sofa_24hours
    FROM vitals_tab v
    LEFT JOIN labs_tab l ON v.stay_id = l.stay_id
    LEFT JOIN scores_tab s ON v.stay_id = s.stay_id
);"""

con.sql(features_query)
df_features = con.sql("SELECT * FROM features_tab").df()
print(f"Extracted {len(df_features):,} features for cohort.")
df_features.head()

: 

In [ ]:
cohort_sql = """
WITH FirstICUStay AS (
    -- Select the minimum ICUSTAY_ID for each SUBJECT_ID to enforce 'First ICU Stay'
    SELECT
        ic.subject_id,
        ic.hadm_id,
        ic.stay_id,
        ic.intime,
        ic.outtime,
        ic.los,
        ROW_NUMBER() OVER (PARTITION BY ic.subject_id ORDER BY ic.intime) AS rn
    FROM
        remote_mimic.mimiciv_icu.icustays ic
),
AdultPatients AS (
    -- Calculate age and filter for adult patients
    SELECT
        fs.*,
        pa.gender,
        (
            (CAST(STRFTIME(fs.intime, '%Y') AS INTEGER) - pa.anchor_year) + pa.anchor_age
        ) AS age_at_admission
    FROM
        FirstICUStay fs
    INNER JOIN 
        remote_mimic.mimiciv_hosp.patients pa ON fs.subject_id = pa.subject_id
    WHERE
        fs.rn = 1 -- Only the first ICU stay
)
SELECT
    ap.subject_id,
    ap.hadm_id,
    ap.stay_id,
    ap.intime,
    ap.outtime,
    ap.los,
    ap.gender,
    ap.age_at_admission AS age,
    -- Target Label: In-Hospital Mortality (IHM)
    adm.hospital_expire_flag AS y_ihm
FROM
    AdultPatients ap
INNER JOIN 
    remote_mimic.mimiciv_hosp.admissions adm ON ap.hadm_id = adm.hadm_id
WHERE
    ap.age_at_admission >= 18 -- Inclusion: Adult patients
AND ap.los * 24 >= 8 -- Exclusion: Minimum length of stay
ORDER BY
    ap.subject_id, ap.intime;
"""

# Execute and store the core cohort data
df_cohort = con.sql(cohort_sql).df()
print(f"Phase 1 Complete. Cohort size: {len(df_cohort):,} patients.")

Phase 1 Complete. Cohort size: 64,363 patients.


In [4]:
static_feat_sql = """
SELECT
    coh.stay_id,
    coh.subject_id,
    coh.age,
    coh.gender,
    coh.y_ihm,
    -- Comorbidities
    cci.charlson_comorbidity_index,
    -- Severity Scores (First 24 Hours - Static)
    o.oasis,
    saps.sapsii,
    -- First Day Vitals (min/max/mean)
    g.gcs_min, -- Example GCS feature
    fvl.heart_rate_min, -- Min Heart Rate in first 24h
    fvl.sbp_max,        -- Max Systolic BP in first 24h
    fvl.resp_rate_mean  -- Mean Respiratory Rate in first 24h
FROM
    df_cohort coh
LEFT JOIN
    remote_mimic.mimiciv_derived.charlson cci ON coh.hadm_id = cci.hadm_id
LEFT JOIN
    remote_mimic.mimiciv_derived.oasis o ON coh.stay_id = o.stay_id
LEFT JOIN
    remote_mimic.mimiciv_derived.sapsii saps ON coh.stay_id = saps.stay_id
LEFT JOIN
    remote_mimic.mimiciv_derived.first_day_gcs g ON coh.stay_id = g.stay_id
LEFT JOIN
    remote_mimic.mimiciv_derived.first_day_vitalsign fvl ON coh.stay_id = fvl.stay_id
LEFT JOIN
    remote_mimic.mimiciv_derived.first_day_lab fdl ON coh.stay_id = fdl.stay_id;
"""

df_static_feat = con.sql(static_feat_sql).df()
print(f"Extracted {len(df_static_feat):,} static, derived feature vectors.")

Extracted 64,363 static, derived feature vectors.


In [6]:
sofa_sql = """
SELECT
    s.stay_id,
    -- Time from ICU intime in hours (already calculated in the derived table)
    s.starttime,
    s.sofa_24hours, -- The total score for the 24 hour period
    s.respiration,
    s.coagulation,
    s.liver,
    s.cardiovascular,
    s.cns,
    s.renal
FROM
    remote_mimic.mimiciv_derived.sofa s
INNER JOIN
    df_cohort coh ON s.stay_id = coh.stay_id
-- Filter to ensure we only get scores calculated during the observation window
WHERE
    s.starttime < coh.intime + INTERVAL '24 hour'
ORDER BY
    s.stay_id, s.starttime;
"""

df_sofa = con.sql(sofa_sql).df()
print(f"Extracted {len(df_sofa):,} dynamic SOFA score entries.")

Extracted 1,475,981 dynamic SOFA score entries.
